In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from math import sin, cos, sqrt, atan2, radians
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


In [2]:
house_address_all = []
house_details_all = []
house_address = []
house_details = []
postal_code_array = []
latitude_array = []
longitude_array = []
year_built_array = []
payment_monthly = []

base_url = 'https://www.remax.ca/ab/calgary-real-estate?page='
urls = [base_url + str(x) for x in range(1,100)]

for url in urls:
    
    #browser.visit(url)
    time.sleep(5)

    # Parse HTML with Beautiful Soup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #time.sleep(5)

    try:
        addresses = soup.find_all('div', class_='left-content flex-one')
        for address in addresses:
            house_address_all.append(address.text)
    except:
        house_address_all.append('None')
        
    #time.sleep(5)
        
    try:
        details = soup.find_all('div', class_='property-details')
        for detail in details:
            house_details_all.append(detail.text)
    except:
        house_details_all.append('None')
        
    #time.sleep(5)
        
    house_details_more=[]
    url_house = []
        
    try:
        for detail in soup.find_all('a'):
            house_details_more.append(detail.get('href'))
    except:
        house_details_more.append('None')
    
    #time.sleep(5)

    for i in house_details_more:
        if '/ab/calgary-real-estate/' in str(i):
            link = 'https://www.remax.ca' + str(i)
            url_house.append(link)
            
    #time.sleep(5)
    
    for i in url_house:
    
        time.sleep(5)

        page = requests.get(i)
        soup = BeautifulSoup(page.text, "html.parser")

        lat_all = soup.find_all('script')[4]

        string = ', '.join(lat_all)
        pc = "postalCode"
        position_pc = string.find(pc)
        pcd = string[(position_pc+14):(position_pc+21)]
        postal_code_array.append(pcd)

        lat = "latitude"
        position_lat = string.find(lat)
        latd = string[(position_lat+11):(position_lat+20)]
        latitude_array.append(latd)

        long = "longitude"
        position_long = string.find(long)
        longd = string[(position_long+12):(position_long+23)]
        longitude_array.append(longd)

        house_address = []

        #time.sleep(5)

        try:
            addresses = soup.find_all('section', class_='property-details')
            for address in addresses:
                house_address.append(address.text)
        except:
            house_address.append('None')

        #time.sleep(5)

        string = ', '.join(house_address)
        s_year = "Year Built"
        position_year = string.find(s_year)
        year_built = string[(position_year+12):(position_year+16)]
        year_built_array.append(year_built)
        
        try:
            payment = soup.find(class_="payment-amount").text
            payment_monthly.append(payment)
        except:
            payment_monthly.append('None')

        #time.sleep(5)

In [3]:
address_df = pd.DataFrame(house_address_all)

new_df = address_df[0].str.split(' ', 2, expand=True)
new_df["price"] = new_df[1].str.replace("$", "")
new_df["price"] = new_df["price"].str.replace(",", "")
new_df["price"] = pd.to_numeric(new_df["price"])

del new_df[0]
del new_df[1]
new_df.head()

,2,price
0,"116 FALTON WAY NE, Calgary, AB, T3J 1K5",334786
1,"239 COPPERPOND BAY SE, Calgary, AB, T2Z 0R3",469900
2,"25 CRESTHAVEN WAY SW, Calgary, AB, T3B 5X8",585000
3,"28 MAHOGANY PK SE, Calgary, AB, T3M 2J9",614900
4,"117 MAHOGANY HTS SE, Calgary, AB, T3M 0X6",478800


In [4]:
final_df_pc = new_df[2].str.split(', Calgary, AB, ', expand=True)
final_df_pc.head()

,0,1
0,116 FALTON WAY NE,T3J 1K5
1,239 COPPERPOND BAY SE,T2Z 0R3
2,25 CRESTHAVEN WAY SW,T3B 5X8
3,28 MAHOGANY PK SE,T3M 2J9
4,117 MAHOGANY HTS SE,T3M 0X6


In [5]:
final_df_pc_clean = final_df_pc[1].str.replace(" ", "")
final_df_pc_clean.head()

0    T3J1K5
1    T2Z0R3
2    T3B5X8
3    T3M2J9
4    T3M0X6
Name: 1, dtype: object

In [6]:
df_add = pd.concat([new_df, final_df_pc_clean, final_df_pc[0]], axis=1)
del df_add[2]
df_add.columns = ["price", "postal_code", "address"]
df_add.head()

,price,postal_code,address
0,334786,T3J1K5,116 FALTON WAY NE
1,469900,T2Z0R3,239 COPPERPOND BAY SE
2,585000,T3B5X8,25 CRESTHAVEN WAY SW
3,614900,T3M2J9,28 MAHOGANY PK SE
4,478800,T3M0X6,117 MAHOGANY HTS SE


In [7]:
details = pd.DataFrame(house_details_all)
details_df_temp = details[0].str.split('|', expand=True)
details_df_temp.head()

,0,1,2,3
0,3 bed,1 + 1 bath,1179 sqft,house
1,3 bed,2 + 1 bath,2136 sqft,house
2,3 bed,2 + 1 bath,1556 sqft,house
3,3 bed,3 + 1 bath,2166 sqft,house
4,4 bed,3 + 1 bath,1703 sqft,house


In [8]:
details_df_bed = details_df_temp[0].str.replace(' bed', '')
details_df_bath = details_df_temp[1].str.replace(' bath', '')
details_df_area = details_df_temp[2].str.replace(' sqft', '')

In [9]:
details_df_bath_all = details_df_bath.str.split('+', expand=True)
details_df_bath_full = details_df_bath_all[0]
details_df_bath_half = details_df_bath_all[1]

In [10]:
details_df_bed = details_df_bed.replace('N/A', np.nan)
details_df_bed = pd.to_numeric(details_df_bed)
details_df_area = details_df_area.replace('N/A', np.nan)
details_df_area = pd.to_numeric(details_df_area)
details_df_bath_full = details_df_bath_full.replace('N/A', np.nan)
details_df_bath_full = pd.to_numeric(details_df_bath_full)
details_df_bath_half = details_df_bath_half.replace('N/A', np.nan)
details_df_bath_half = pd.to_numeric(details_df_bath_half)

In [11]:
data = {'bed':details_df_bed, 'full_bath':details_df_bath_full, 'half_bath':details_df_bath_half,
       'property_area':details_df_area, 'property_type':details_df_temp[3]}

In [12]:
details_df = pd.DataFrame(data)
details_df.head()

,bed,full_bath,half_bath,property_area,property_type
0,3.0,1.0,1.0,1179.0,house
1,3.0,2.0,1.0,2136.0,house
2,3.0,2.0,1.0,1556.0,house
3,3.0,3.0,1.0,2166.0,house
4,4.0,3.0,1.0,1703.0,house


In [13]:
calgary_df_dup = pd.concat([df_add, details_df], axis=1)
calgary_df = calgary_df_dup.drop_duplicates()
calgary_df.head()

,price,postal_code,address,bed,full_bath,half_bath,property_area,property_type
0,334786,T3J1K5,116 FALTON WAY NE,3.0,1.0,1.0,1179.0,house
1,469900,T2Z0R3,239 COPPERPOND BAY SE,3.0,2.0,1.0,2136.0,house
2,585000,T3B5X8,25 CRESTHAVEN WAY SW,3.0,2.0,1.0,1556.0,house
3,614900,T3M2J9,28 MAHOGANY PK SE,3.0,3.0,1.0,2166.0,house
4,478800,T3M0X6,117 MAHOGANY HTS SE,4.0,3.0,1.0,1703.0,house


## Cleanup Postal Codes, Latitude, Longitude and Year Built

In [14]:
postal_codes_messy = pd.DataFrame(postal_code_array, columns=["postal_code"])

In [15]:
postal_codes_messy.head()

,postal_code
0,"T3J1K5"""
1,"T2Z0R3"""
2,"T3B5X8"""
3,"T3M2J9"""
4,"T3M0X6"""


In [16]:
postal_code_array_clean = postal_codes_messy["postal_code"].str.replace('"', '')
postal_code_array_clean = postal_code_array_clean.str.replace(" ", "")

In [17]:
latitude_array_messy = pd.DataFrame(latitude_array, columns=["latitude"])

In [18]:
latitude_array_messy = latitude_array_messy["latitude"].str.replace(',', '')
latitude_array_messy.head()

0    51.102699
1    50.919753
2    51.085292
3    50.887413
4    50.903714
Name: latitude, dtype: object

In [19]:
longitude_array_messy = pd.DataFrame(longitude_array, columns=["longitude"])

In [20]:
longitude_array_messy = longitude_array_messy["longitude"].str.replace(',', '')
longitude_array_messy.head()

0    -113.942978
1    -113.921865
2    -114.264517
3    -113.940111
4    -113.934113
Name: longitude, dtype: object

<div class="alert alert-danger">
    <b>Unused</b>
    
year_built_messy = pd.DataFrame(year_built_array, columns=["year_built"])<br>
year_built_clean = year_built_messy["year_built"].str.replace('', np.nan)<br>
year_built_num = year_built_messy.astype(int)

year_built_num.head()
</div>

In [21]:
year_built_messy = pd.DataFrame(year_built_array, columns=["year_built"])
#year_built_messy = pd.to_numeric(year_built_messy)
year_built_messy.head()

,year_built
0,1980
1,2011
2,2002
3,2017
4,2011


In [22]:
year_built_clean = year_built_messy["year_built"].replace('', np.nan)

In [23]:
year_built_clean.head(10)

0    1980
1    2011
2    2002
3    2017
4    2011
5    2020
6    1970
7    2006
8    1911
9    2021
Name: year_built, dtype: object

In [54]:
year_built_digits = []

for i in year_built_clean:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        #year_built_clean.replace(i, np.nan)
        year_built_digits.append(np.nan)
    if ((str(i) == ' 9 2') or (str(i) == ' 8 2') or (str(i) == ' 2 2') or (str(i) == 'nasq') or (str(i) == ' 6 R') or (str(i) == ' 6 S')):
        year_built_digits.append(np.nan)
    else:
        year_built_digits.append(i)

In [55]:
year_built_digits[:10]

['1980',
 '2011',
 '2002',
 '2017',
 '2011',
 '2020',
 '1970',
 '2006',
 '1911',
 '2021']

In [56]:
year_built_num = pd.to_numeric(year_built_digits)
year_built_num[:10]

array([1980., 2011., 2002., 2017., 2011., 2020., 1970., 2006., 1911.,
       2021.])

In [57]:
year_built_num_pd = pd.DataFrame(year_built_num)

In [58]:
years_old = 2020 - year_built_num
years_old[:10]

array([ 40.,   9.,  18.,   3.,   9.,   0.,  50.,  14., 109.,  -1.])

In [59]:
years_old_df = pd.DataFrame(years_old)

In [60]:
latitude_digits = []

for i in latitude_array_messy:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        latitude_digits.append(np.nan)
    else:
        latitude_digits.append(i)

In [61]:
longitude_digits = []

for i in longitude_array_messy:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        longitude_digits.append(np.nan)
    else:
        longitude_digits.append(i)

In [62]:
latitude_digits = pd.to_numeric(latitude_digits)
longitude_digits = pd.to_numeric(longitude_digits)

In [63]:
payment_monthly_messy = pd.DataFrame(payment_monthly, columns=["monthly_payments"])
payment_monthly_messy.head()

,monthly_payments
0,$1447.65
1,$2031.91
2,$2529.61
3,$2658.90
4,$2070.39


In [64]:
payment_monthly_messy = payment_monthly_messy["monthly_payments"].str.replace('$', '')
payment_monthly_messy.head(10)

0    1447.65
1    2031.91
2    2529.61
3    2658.90
4    2070.39
5    2594.43
6    1167.51
7    2161.63
8    2486.37
9    1642.73
Name: monthly_payments, dtype: object

In [65]:
payment_monthly_messy = payment_monthly_messy.replace('', np.nan)

In [66]:
payment_monthly_clean = payment_monthly_messy.replace('None', np.nan)
payment_monthly_clean = pd.to_numeric(payment_monthly_clean)

<div class="alert alert-success">
    <b>Guide</b>
    
https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

#### approximate radius of earth in km
R = 6373.0
    
lat1 = radians(52.2296756)<br>
lon1 = radians(21.0122287)

lat2 = radians(52.406374)<br>
lon2 = radians(16.9251681)

dlon = lon2 - lon1<br>
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2<br>
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
    
#### Calgary Downtown coordinates (51.0449485,-114.0739559)
</div>

In [67]:
distance_downtown = []

for i, j in zip(latitude_digits, longitude_digits):
    lat1 = radians(i)
    lon1 = radians(j)
    lat2 = radians(51.0449485)
    lon2 = radians(-114.0739559)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6373.0
    distance = (R * c)
    
    distance_downtown.append(distance)

In [68]:
distance_downtown[:5]

[11.18273182105422,
 17.531321901647406,
 14.055942946629264,
 19.87334043827771,
 18.51259582066417]

In [69]:
distance_downtown_df = pd.DataFrame(distance_downtown)

<div class="alert alert-warning">
    <b>Another example</b>
    
import geopy.distance

coords_1 = (52.2296756, 21.0122287)<br>
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.vincenty(coords_1, coords_2).km)
</div>

In [70]:
more_details = pd.DataFrame([latitude_digits, longitude_digits, year_built_num]).T
#more_details.columns = ["postal_code", "latitude", "longitude", "year_built"]
more_details.head()

,0,1,2
0,51.102699,-113.942978,1980.0
1,50.919753,-113.921865,2011.0
2,51.085292,-114.264517,2002.0
3,50.887413,-113.940111,2017.0
4,50.903714,-113.934113,2011.0


In [71]:
more_df = pd.concat([postal_code_array_clean, more_details, years_old_df, distance_downtown_df, payment_monthly_clean], axis=1)
more_df.columns = ["postal_code", "latitude", "longitude", "year_built", "years_old", "distance_downtown", "monthly_payments"]
more_df.head()

,postal_code,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments
0,T3J1K5,51.102699,-113.942978,1980.0,40.0,11.182732,1447.65
1,T2Z0R3,50.919753,-113.921865,2011.0,9.0,17.531322,2031.91
2,T3B5X8,51.085292,-114.264517,2002.0,18.0,14.055943,2529.61
3,T3M2J9,50.887413,-113.940111,2017.0,3.0,19.873340,2658.90
4,T3M0X6,50.903714,-113.934113,2011.0,9.0,18.512596,2070.39


In [72]:
result_dup = pd.merge(calgary_df, more_df, on='postal_code')
result = result_dup.drop_duplicates()
result = result.reset_index(drop=True)
result.head()

,price,postal_code,address,bed,full_bath,half_bath,property_area,property_type,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments
0,334786,T3J1K5,116 FALTON WAY NE,3.0,1.0,1.0,1179.0,house,51.102699,-113.942978,1980.0,40.0,11.182732,1447.65
1,469900,T2Z0R3,239 COPPERPOND BAY SE,3.0,2.0,1.0,2136.0,house,50.919753,-113.921865,2011.0,9.0,17.531322,2031.91
2,585000,T3B5X8,25 CRESTHAVEN WAY SW,3.0,2.0,1.0,1556.0,house,51.085292,-114.264517,2002.0,18.0,14.055943,2529.61
3,614900,T3M2J9,28 MAHOGANY PK SE,3.0,3.0,1.0,2166.0,house,50.887413,-113.940111,2017.0,3.0,19.873340,2658.90
4,478800,T3M0X6,117 MAHOGANY HTS SE,4.0,3.0,1.0,1703.0,house,50.903714,-113.934113,2011.0,9.0,18.512596,2070.39


In [73]:
result.to_csv('data/calgary_listings-1-100.csv', index=False)

-----------

<div class="alert alert-info">
    <b>UNSED</b>
</div>

In [ ]:
house_details_more=[]
url_house = []
try:
    for detail in soup.find_all('a'):
        house_details_more.append(detail.get('href'))
except:
    house_details_more.append('None')
    
for i in house_details_more:
    if '/ab/calgary-real-estate/' in str(i):
        link = 'https://www.remax.ca' + str(i)
        url_house.append(link)
url_house

In [ ]:
url_house[:5]

In [ ]:
PostalCode = []
Latitude = []
Longitude = []
YearBuilt = []

for i in url_house[:5]:
    
    time.sleep(5)
    
    page = requests.get(i)
    soup = BeautifulSoup(page.text, "html.parser")
    
    lat_all = soup.find_all('script')[4]
    
    string = ', '.join(lat_all)
    pc = "postalCode"
    position_pc = string.find(pc)
    pcd = string[(position_pc+14):(position_pc+21)]
    PostalCode.append(pcd)

    lat = "latitude"
    position_lat = string.find(lat)
    latd = string[(position_lat+11):(position_lat+20)]
    Latitude.append(latd)

    long = "longitude"
    position_long = string.find(long)
    longd = string[(position_long+12):(position_long+23)]
    Longitude.append(longd)
    
    house_address = []
    
    time.sleep(5)
    
    try:
        addresses = soup.find_all('section', class_='property-details')
        for address in addresses:
            house_address.append(address.text)
    except:
        house_address.append('None')
    
    time.sleep(5)
    
    string = ', '.join(house_address)
    s_year = "Year Built"
    position_year = string.find(s_year)
    year_built = string[(position_year+12):(position_year+16)]
    YearBuilt.append(year_built)